API 통해서 가져오면 쉽게 가져올 수 있다.
일반적으로 데이터 요청하면 응답값이 json으로 온다.
왜 배워야되냐?
외부데이터를 시스템화 해서 데이터베이스에 넣고 싶을 때.
실시간, 공공, social media 가져올 때 신속하게 가져올 때 많이 씀.

준수 튜터님이 쓰시는 https://taengsweb.tistory.com/56

### 숙제

- template.env 를 .env로 변환하고 API_KEY 값을 채워주세요
- OpenWeather API 회원가입 후 받은 API_KEY로 sydney에 대한 다른 정보도 출력하기 
(https://openweathermap.org/api) 

https://openweathermap.org/current

api 설명 - Built-in API request by city name

### 01. 패키지 불러오기

In [2]:
import requests
import pandas as pd
from dotenv import load_dotenv
import os
import json
load_dotenv()

# code here 

True

### 02. Requests 를 통해 웹사이트 HTML 가져오기

In [3]:
# OpenWeather API 요청을 위한 기본 설정
api_key = os.environ.get('API_KEY')
city_name = "Sydney"
url = f'https://api.openweathermap.org/data/2.5/weather?q={city_name}&units=metric&appid={api_key}'
response = requests.get(url)
print(response.status_code)

200


### 03. Python 딕셔너리를 통해 HTML 파싱 후 출력

In [4]:
data = response.json()
print(json.dumps(data, indent=4)) #indentation = 들여쓰기

{
    "coord": {
        "lon": 151.2073,
        "lat": -33.8679
    },
    "weather": [
        {
            "id": 803,
            "main": "Clouds",
            "description": "broken clouds",
            "icon": "04d"
        }
    ],
    "base": "stations",
    "main": {
        "temp": 20.52,
        "feels_like": 20.4,
        "temp_min": 19.61,
        "temp_max": 22.08,
        "pressure": 1021,
        "humidity": 68,
        "sea_level": 1021,
        "grnd_level": 1013
    },
    "visibility": 10000,
    "wind": {
        "speed": 5.66,
        "deg": 130
    },
    "clouds": {
        "all": 75
    },
    "dt": 1731991933,
    "sys": {
        "type": 2,
        "id": 2046014,
        "country": "AU",
        "sunrise": 1731955287,
        "sunset": 1732005597
    },
    "timezone": 39600,
    "id": 2147714,
    "name": "Sydney",
    "cod": 200
}


In [5]:
# 테이블화
pd.json_normalize(data)

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.sea_level,main.grnd_level,wind.speed,wind.deg,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 803, 'main': 'Clouds', 'description': ...",stations,10000,1731991933,39600,2147714,Sydney,200,151.2073,-33.8679,...,1021,1013,5.66,130,75,2,2046014,AU,1731955287,1732005597


In [6]:
df_weather = pd.json_normalize(data)
df_weather.columns

Index(['weather', 'base', 'visibility', 'dt', 'timezone', 'id', 'name', 'cod',
       'coord.lon', 'coord.lat', 'main.temp', 'main.feels_like',
       'main.temp_min', 'main.temp_max', 'main.pressure', 'main.humidity',
       'main.sea_level', 'main.grnd_level', 'wind.speed', 'wind.deg',
       'clouds.all', 'sys.type', 'sys.id', 'sys.country', 'sys.sunrise',
       'sys.sunset'],
      dtype='object')

In [7]:
# Time of data calculation, unix, UTC
df_weather['dt']

0    1731991933
Name: dt, dtype: int64

pd.to_datetime(df, unit) 에서</br>
unit 매개변수는 입력된 값(df_weather["dt"])이 어떤 단위인지 Pandas에게 알려줍니다.

- unit="s": 입력값을 초 단위로 처리합니다.
- unit="ms": 입력값을 밀리초 단위로 처리합니다.
- unit="us" 또는 unit="μs": 입력값을 마이크로초 단위로 처리합니다.
- unit="ns": 입력값을 나노초 단위로 처리합니다.

In [8]:
pd.to_datetime(df_weather['dt'], unit="s")

0   2024-11-19 04:52:13
Name: dt, dtype: datetime64[ns]

In [9]:
df_weather.T

,0
weather,"[{'id': 803, 'main': 'Clouds', 'description': ..."
base,stations
visibility,10000
dt,1731991933
timezone,39600
id,2147714
name,Sydney
cod,200
coord.lon,151.2073
coord.lat,-33.8679


In [10]:
df_weather['measured_at'] = pd.to_datetime(df_weather["dt"], unit='s')
df_weather['dt'] = df_weather['measured_at'].dt.strftime('%Y%m%d') #기준년월일 (YYYYMMDD)
df_weather['time'] = df_weather['measured_at'].dt.strftime('%H%M%S') #기준년월일 (HHHHMMSS) 
df_weather.T

,0
weather,"[{'id': 803, 'main': 'Clouds', 'description': ..."
base,stations
visibility,10000
dt,20241119
timezone,39600
id,2147714
name,Sydney
cod,200
coord.lon,151.2073
coord.lat,-33.8679


unit="s"는 Pandas의 pd.to_datetime() 함수에서 사용되는 옵션으로, 시간 데이터를 초(second) 단위의 유닉스 타임스탬프로 해석하도록 지정합니다.

In [11]:
df_selected = df_weather[["dt", "time", "measured_at", "id", "name", "main.temp", "main.humidity", "wind.speed"]]
df_selected

,dt,time,measured_at,id,name,main.temp,main.humidity,wind.speed
0,20241119,045213,2024-11-19 04:52:13,2147714,Sydney,20.52,68,5.66


In [12]:
df_selected = df_selected.rename(#컬럼명 수정
    columns={"name": "city", "main.temp": "temp", "main.humidity": "humid", "wind.speed": "wind_speed"}
    )

### 04. Pandas 로 CSV 파일 저장

In [13]:
os.getcwd()

'/Users/jun/GitStudy/Data_4/src/challenge/week2'

In [14]:
# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(df_selected)
df.to_csv('weather_api.csv', index=False)
print("날씨 정보가 CSV 파일로 저장되었습니다.")

날씨 정보가 CSV 파일로 저장되었습니다.


### 05. Class로 만들어보자 

In [15]:
import requests

class WeatherApiClient:
    """
    OpenWeatherMap API 클라이언트를 사용하여 날씨 데이터를 가져오는 클래스입니다.
    """

    def __init__(self, api_key) -> None:
        self.base_url = 'https://api.openweathermap.org/data/2.5'
        if api_key is None:
            # Python의 raise 키워드는 예외(Exception)를 명시적으로 발생시킬 때 사용됩니다. 
            # 프로그램 실행 중 특정 조건에서 오류를 발생시키고, 예외 처리 로직으로 제어를 넘길 때 사용
            raise Exception("None API key")
        self.api_key = api_key
    
    # -> dict : Return Type Hint
    def get_city(self, city_name: str, temperature_units: str = 'metric') -> dict:
        """
        지정 도시의 최신 날씨 데이터 가져옵니다.

        parameters:
        - city_name (str) : 날씨 정보르 조회할 도시 이름
        - temperature_units (str) : 온도 단위 (기본값은 'metric'으로 섭씨)
                    Unit Default: Kelvin, Metric: Celsius, Imperial: Fahrenheit

        Returns:
        - dict : 요청 날씨 데이터 포함된 JSON 응답을 반환

        Raises:
        - Exception: API 요청이 실패한 경우 상태 코드와 응답 메세지와 함께 예외가 발생합니다.
        """

        params = {"q":city_name, "units":temperature_units, "appid":self.api_key}
        response = requests.get(f"{self.base_url}/weather",params=params)
        if response.status_code == 200:
            return response.json()
        else :
            raise Exception(
                f"Open Weather API에서 데이터를 추출하지 못했습니다. 상태 코드: {response.status_code}. 응답: {response.text}"
            )

In [17]:
import pandas as pd
from dotenv import load_dotenv
import os
load_dotenv()

#load_dotenv(): python-dotenv 라이브러리를 사용해 
#.env 파일에 정의된 환경 변수를 os.environ에 로드합니다.
weather_api_client = WeatherApiClient(api_key= os.environ.get("API_KEY"))
cities = ['Tokyo','Osaka','Sydney','Seoul','Busan','Singapore']

data = list()
for c in cities:
    data.append(weather_api_client.get_city(c))

# 간단한 전처리
df_weather = pd.json_normalize(data) # JSON 데이터를 Pandas 형태로 변환 
df_weather['measured_at'] = pd.to_datetime(df_weather['dt'], unit='s') # 기준시간 
df_weather['dt'] = df_weather['measured_at'].dt.strftime('%Y%m%d') # 기준년월일 (YYYYMMDD) 
df_weather['time'] = df_weather['measured_at'].dt.strftime('%H%M%S') # 기준년월일 (HHHHMMSS) 
df_selected = df_weather[["dt","time","measured_at","id","name", "main.temp", "main.humidity", "wind.speed"]]
df_selected = df_selected.rename( # 컬럼명 수정 
    columns={"name": "city", "main.temp": "temperature", "main.humidity": "humidity", "wind.speed": "wind_speed"}
)

# DataFrame 생성 및 CSV 파일로 저장
df = pd.DataFrame(df_selected)
df.to_csv('weather_api.csv', index=False)

In [18]:
df

,dt,time,measured_at,id,city,temperature,humidity,wind_speed
0,20241119,044823,2024-11-19 04:48:23,1850144,Tokyo,12.50,37,5.66
1,20241119,045726,2024-11-19 04:57:26,1853909,Osaka,11.51,54,5.66
2,20241119,045213,2024-11-19 04:52:13,2147714,Sydney,20.52,68,5.66
3,20241119,045236,2024-11-19 04:52:36,1835848,Seoul,7.19,53,2.57
4,20241119,045414,2024-11-19 04:54:14,1838524,Busan,12.99,50,4.63
5,20241119,044924,2024-11-19 04:49:24,1880252,Singapore,27.69,83,3.09
